<a href="https://colab.research.google.com/github/maksimyaromin/retrospectra/blob/main/Retrospectra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [95]:
import tensorflow as tf
import numpy as np

In [11]:
data_dir = '/content/drive/MyDrive/retrospectra/data'

In [22]:
train_generator = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split = 0.2,
  subset = "training",
  seed = 1337,
  image_size = (244, 244),
  batch_size = 32,
)

Found 80 files belonging to 2 classes.
Using 64 files for training.


In [23]:
validation_dataset = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split = 0.2,
  subset = "validation",
  seed = 1337,
  image_size = (244, 244),
  batch_size = 32,
)

Found 80 files belonging to 2 classes.
Using 16 files for validation.


In [96]:
class_names = train_generator.class_names

print(class_names)
len(class_names)

['France', 'Russia']


2

In [57]:
base_model = tf.keras.applications.resnet50.ResNet50(
  include_top = False,
  input_shape = (244, 244, 3),
  weights = "imagenet"
)
base_model.trainable = False

In [18]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 244, 244, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 250, 250, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 122, 122, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 122, 122, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [97]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(256, activation = "relu"),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(len(class_names), activation = "softmax")
])

In [76]:
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 global_average_pooling2d_1  (None, 2048)              0         
 1 (GlobalAveragePooling2D)                                      
                                                                 
 dense_22 (Dense)            (None, 256)               524544    
                                                                 
 dropout_11 (Dropout)        (None, 256)               0         
                                                                 
 dense_23 (Dense)            (None, 2)                 514       
                                                                 
Total params: 24112770 (91.98 MB)
Trainable params: 525058 (2.00 MB)
Non-trainable params: 23587712 (89.98 MB)
________

In [83]:
len(model.trainable_variables)

4

In [98]:
model.compile(
  optimizer = tf.keras.optimizers.Adam(0.0001),
  loss = "sparse_categorical_crossentropy",
  metrics = ["accuracy"]
)

In [88]:
loss0, accuracy0 = model.evaluate(validation_dataset)

print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

1/1 [==============================] - 4s 4s/step - loss: 0.8516 - accuracy: 0.4375
initial loss: 0.85
initial accuracy: 0.44


In [99]:
history = model.fit(
  train_generator,
  epochs = 10,
  validation_data = validation_dataset
)

Epoch 1/10
2/2 [==============================] - 16s 9s/step - loss: 0.9869 - accuracy: 0.5000 - val_loss: 0.6273 - val_accuracy: 0.7500
Epoch 2/10
2/2 [==============================] - 12s 8s/step - loss: 0.9904 - accuracy: 0.4844 - val_loss: 0.6604 - val_accuracy: 0.6250
Epoch 3/10
2/2 [==============================] - 12s 7s/step - loss: 0.6537 - accuracy: 0.6406 - val_loss: 0.6090 - val_accuracy: 0.7500
Epoch 4/10
2/2 [==============================] - 13s 8s/step - loss: 0.5699 - accuracy: 0.6250 - val_loss: 0.5553 - val_accuracy: 0.8750
Epoch 5/10
2/2 [==============================] - 13s 8s/step - loss: 0.6224 - accuracy: 0.7500 - val_loss: 0.5421 - val_accuracy: 0.8125
Epoch 6/10
2/2 [==============================] - 13s 8s/step - loss: 0.8686 - accuracy: 0.5938 - val_loss: 0.5276 - val_accuracy: 0.8125
Epoch 7/10
2/2 [==============================] - 14s 9s/step - loss: 0.6460 - accuracy: 0.7031 - val_loss: 0.5109 - val_accuracy: 0.8125
Epoch 8/10
2/2 [==================

In [134]:
image_url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/f/ff/Louis_nicolas_davout.jpg/600px-Louis_nicolas_davout.jpg'
image_path = tf.keras.utils.get_file('Image9', origin = image_url)

img = tf.keras.utils.load_img(
  image_path,
  target_size = (244, 244)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)


117250/117250 [==============================] - 0s 1us/step


In [135]:
predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

1/1 [==============================] - 0s 202ms/step


In [136]:
print("This image most likely belongs to {}.".format(class_names[np.argmax(score)]))

This image most likely belongs to France.
